## compute keypoints and descriptors

In [32]:
def read(path):
    img= cv2.imread(path)
    gray= cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
#     cv2.imshow("Original image",gray)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    sift= cv2.xfeatures2d.SIFT_create()
#     sift= cv2.SIFT()
#     keypoints= sift.detect(gray,None)
    keypoints, descriptors=sift.detectAndCompute(gray,None)
    draw_keypoints(img, keypoints, gray, path)
    return keypoints, descriptors, img

## drawing keypoints for respective images sent 

In [42]:
def draw_keypoints(color_image, keypoints, gray_image, og):
    color_image=cv2.drawKeypoints(color_image, keypoints,gray_image,color=(0,0,0))
    cv2.imwrite(og+"task1_sift1.jpg", color_image)
    cv2.imshow("key points", color_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

## warp and stitch the images together task 1.5

In [33]:
def warpandstitch(img1, img2, H):
    h1,w1 = img1.shape[:2]
    h2,w2 = img2.shape[:2]
    a = np.float32([[0,0],[0,h1],[w1,h1],[w1,0]]).reshape(-1,1,2)
    b = np.float32([[0,0],[0,h2],[w2,h2],[w2,0]]).reshape(-1,1,2)
    b_ = cv2.perspectiveTransform(b, H)
    pts = np.concatenate((a, b_), axis=0)
    [xmin, ymin] = np.int32(pts.min(axis=0).ravel() - 0.5)
    [xmax, ymax] = np.int32(pts.max(axis=0).ravel() + 0.5)
    t = [-xmin,-ymin]
    Ht = np.array([[1,0,t[0]],[0,1,t[1]],[0,0,1]]) # translate

    result = cv2.warpPerspective(img2, Ht.dot(H), (xmax-xmin, ymax-ymin))
    result[t[1]:h1+t[1],t[0]:w1+t[0]] = img1
    return result

## matching the pixels in 2 images

In [35]:
def match(keypoints1, descriptor1, keypoints2, descriptor2):
    comp_neighbour=2
    matcher= cv2.BFMatcher()
    matches= matcher.knnMatch(descriptor1,descriptor2, comp_neighbour)
    
    good_match=[]
    
    for m,n in matches:
        if(m.distance<0.75*(n.distance)):
            good_match.append(m)
    
    return good_match
        

In [36]:
def draw_matches( image1, image2, keypoint1,keypoint2, matches):
    
    result_knn= cv2.drawMatches(image1,keypoint1,image2,keypoint2,matches,None,flags=2,matchColor=(0,0,0))
    cv2.imwrite("task1_matches_knn.jpg", result_knn)
    cv2.imshow("result ", result_knn )
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return result_knn

## COMPUTING HOMOGRAPHY MATRIX

In [37]:
def homography(goodmatches, keypoints1,keypoints2):
    image1=np.array([keypoints1[m.queryIdx].pt for m in goodmatches]).reshape(-1,1,2)
    image2=np.array([keypoints2[m.trainIdx].pt for m in goodmatches]).reshape(-1,1,2)
    
    M, mask=cv2.findHomography(image1,image2, cv2.RANSAC, 8)
    return M, mask
    
    
    
    

## task 1.4

In [38]:
def task_1_4(mask, image1, image2, keypoints1,keypoints2, good_match):
    
        
    mask2=[]
    good_matches2=[]
    for _ in range(10):
        j=np.random.randint(len(mask))# for displaying only 10 matches
        if (1==mask[j]):
            mask2.append(mask[j])
            good_matches2.append(good_match[j]) 
    
    var= dict(matchColor=(255,0,0),singlePointColor=None, matchesMask=mask2, flags=2)
    
    
    img=cv2.drawMatches(image1,keypoints1,image2, keypoints2, good_matches2, None, **var) #good_matches for displaying 
    cv2.imwrite("task1_matches.jpg",img)                                                   #only 10 matches
    cv2.imshow("task1.4",img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return img

## MAIN FUNCTION

In [40]:
def main():
    source1="mountain1.jpg"
    source2="mountain2.jpg"
    keypoints1, descriptor1, img1 =read(source1)  
    keypoints2, descriptor2, img2 =read(source2)
#     print(img2.shape[1]) # image is actually 517 x 374
#     print(descriptor1.shape)  @(1027 x 128) -> shape of descriptors is (number of keypoints x 128).   
    
    good_matches=match(keypoints1,descriptor1,keypoints2,descriptor2)
#     print(len(good_matches)) @ 244 matches
    result_knn=draw_matches(img1,img2,keypoints1,keypoints2,good_matches)
    
    homography_matrix, mask=homography(good_matches, keypoints1,keypoints2)
    print("--------------HOMOGRAPHY MATRIX--------------")
    print(homography_matrix)
    mask1=mask.ravel().tolist()
 
    task_1_4(mask1,img1,img2,keypoints1,keypoints2, good_matches)
    
    result= warpandstitch(img2,img1,homography_matrix)

    cv2.imshow("warped image", result)
    cv2.waitKey(0)
    cv2.imwrite("task1_pano.jpg", result)
    cv2.destroyAllWindows()

In [41]:
if __name__=="__main__":
    import cv2 
    from matplotlib import pyplot as plt
    import numpy as np
    UBIT = 'mkulshre'
    np.random.seed(sum([ord(c) for c in UBIT]))
    main()

--------------HOMOGRAPHY MATRIX--------------
[[ 1.58720383e+00 -2.91747487e-01 -3.95226577e+02]
 [ 4.48097730e-01  1.43063317e+00 -1.90273581e+02]
 [ 1.20808474e-03 -6.07786039e-05  1.00000000e+00]]
